In [94]:
import os
import json
import pickle

from collections import Counter

import pandas as pd
import numpy as np

# Setup

In [95]:
data_dir = os.path.join(os.curdir, "data")

raw_dataset_url = "https://nlp-slides.vercel.app/clean-tweets.tsv"

In [96]:
raw = pd.read_csv(filepath_or_buffer=raw_dataset_url, sep="\t")

raw.head()

,Tweet,Country,Topic,Sentiment,Sentiment_Expression,Sentiment_Target,word_count,char_count,clean_text,clean_stemmed
0,"""أنا أؤمن بأن الانسان ينطفئ جماله عند ابتعاد م...",lebanon,personal,negative,implicit,بريق العيون,23,132,اومن بان الانسان ينطفي جماله ابتعاد يحب بريق ا...,اوم بان انس نطف جمل بعد يحب برق عين خفي صبح ذب...
1,من الذاكره... @3FInQe . عندما اعتقد كريستيانو ...,jordan,sports,positive,explicit,افضل لاعب في العالم,23,141,الذاكره عندما اعتقد كريستيانو انه افضل لاعب ال...,ذكر عند عقد كريستيانو انه فضل لعب علم ككا يسي ...
2,لا نخلو من ضغوطات الحياة. فنحن نعيش على أرض أع...,palestine,personal,neutral,none,none,24,133,نخلو ضغوطات الحياه فنحن نعيش ارض اعدت للبلاء و...,خلو ضغط حيه فنح نعش ارض اعد بلء ولم سلم بيء وك...
3,#مصطلحات_لبنانيه_حيرت_البشريه بتوصل عالبيت ، ب...,lebanon,personal,negative,explicit,مصطلحات_لبنانيه,23,135,بتوصل عالبيت بنط بقلك جيت بتقعد لتتحدث معو بقل...,وصل علب بنط بقل جيت قعد حدث معو بقل شو تقم تمش...
4,نصمت !! لتسير حياتنا على مً يرام فالناّس لم تع...,palestine,personal,negative,explicit,س لم تعد كما ك,16,67,نصمت لتسير حياتنا يرام فالناس تعد كانت نقيه,نصم تسر حيت يرم لنس تعد كانت نقه


In [97]:
tweets = raw["clean_stemmed"]

# Tokenization

First step is to split the dataset into small bits, each bit is called a `token`

once the corpus is tokenized, we can assign each unique `token` an `index`, note that the index value for any token is not important 

In [98]:
sample = tweets.iloc[0]
sample

'اوم بان انس نطف جمل بعد يحب برق عين خفي صبح ذبل نطف تحل ربع خرف'

Tokenizing have many forms, examples: 
- Word Level Tokenization
- Character Level Tokenization
- Sub-word Tokenization
- Byte Pair Encoding

In this chapter we will cover word level and character level tokenization, others will be covered in later chapters

In [99]:
# Implement a generic Tokenizer class
# Different tokenizers will inherit from this class
class Tokenizer:
    def __init__(self, corpus: list[str], min_frequency: int = None):
        self.min_frequency = min_frequency
        self.vocab = self._create_vocab(corpus=corpus)
        
        
    def _create_vocab(self, corpus: list[str]) -> dict[str, int]:
        ...
    
    def _tokenize_document(self, document: str) -> list[int]:
        ...
    def tokenize(self, documents: list[str]) -> list[list[int]]:
        return [self._tokenize_document(document) for document in documents]
    
    def __len__(self):
        return self.vocab

## Word Level

Here we split the sentence into indivual words, omitting the whitespaces between them

In [100]:
sample_tokens = sample.split(" ")
sample_tokens

['اوم',
 'بان',
 'انس',
 'نطف',
 'جمل',
 'بعد',
 'يحب',
 'برق',
 'عين',
 'خفي',
 'صبح',
 'ذبل',
 'نطف',
 'تحل',
 'ربع',
 'خرف']

In [101]:
# Create a list of all unique tokens in the corpus
all_word_tokens = [token for sample in tweets for token in sample.split(" ")]

word_level_tokens = set(all_word_tokens)
print(len(word_level_tokens), len(all_word_tokens))

6420 62252


In [102]:
word2idx = {token: index for index, token in enumerate(word_level_tokens)}
word2idx

{'': 0,
 'صرن': 1,
 'شمع': 2,
 'ظبي': 3,
 'عضمر': 4,
 'نكم': 5,
 'صومال': 6,
 'لهذا': 7,
 'لفق': 8,
 'مقربينفلا': 9,
 'بشر': 10,
 'باص': 11,
 'شلء': 12,
 'حوت': 13,
 'حمص': 14,
 'ابب': 15,
 'طير': 16,
 'جيي': 17,
 'حتل': 18,
 'فضح': 19,
 'لتك': 20,
 'كدش': 21,
 'بو': 22,
 'صوج': 23,
 'يضع': 24,
 'زوي': 25,
 'جهلااء': 26,
 'يضر': 27,
 'كشف': 28,
 'بحظ': 29,
 'كست': 30,
 'اذنب': 31,
 'يبا': 32,
 'جلب': 33,
 'خلج': 34,
 'شسع': 35,
 'سامحنييارب': 36,
 'وخد': 37,
 'رسيل': 38,
 'اغل': 39,
 'رفع': 40,
 'لتع': 41,
 'نهز': 42,
 'زدحم': 43,
 'تدل': 44,
 'رحمو': 45,
 'وبج': 46,
 'رشهعطر': 47,
 'لعق': 48,
 'ايا': 49,
 'شيب': 50,
 'انسانسياتي': 51,
 'يعز': 52,
 'حضن': 53,
 'منج': 54,
 'هيد': 55,
 'وعو': 56,
 'مفاتيح': 57,
 'ساو': 58,
 'نحن': 59,
 'فحب': 60,
 'ماتقول': 61,
 'برك': 62,
 'اشخ': 63,
 'ادش': 64,
 'يتذ': 65,
 'كجزء': 66,
 'نحد': 67,
 'هددو': 68,
 '😐': 69,
 '💤': 70,
 'يذء': 71,
 'جيه': 72,
 'ايجر': 73,
 'وصي': 74,
 'عبدالعزيز': 75,
 'لكع': 76,
 'نزف': 77,
 'فعم': 78,
 '🏿': 79,
 'ناب': 80,

In [103]:
class WordLevelTokenizer(Tokenizer):
    def __init__(self, corpus: list[str], min_frequency: int = 0):
        super().__init__(corpus=corpus, min_frequency=min_frequency)
        
    def _create_vocab(self, corpus: list[str]) -> dict[str, int]:
        tokens_counter = Counter([token for sample in corpus for token in sample.split(" ")])
        tokens = [token for token, count in tokens_counter.items() if count >= self.min_frequency]
        vocab = {token: index for index, token in enumerate(tokens, start=2)} 
        vocab["[PAD]"] = 0
        vocab["[OOV]"] = 1
        return vocab
    
    def _tokenize_document(self, document: str) -> list[int]:
        return [self.vocab.get(token, -1) for token in document.split(" ")]

    

In [104]:
word_level_tokenizer = WordLevelTokenizer(corpus=tweets, min_frequency=10)

In [105]:
word_level_tokenizer.vocab

{'بان': 2,
 'انس': 3,
 'جمل': 4,
 'بعد': 5,
 'يحب': 6,
 'عين': 7,
 'صبح': 8,
 'تحل': 9,
 'ربع': 10,
 'خرف': 11,
 'ذكر': 12,
 'عند': 13,
 'عقد': 14,
 'انه': 15,
 'فضل': 16,
 'لعب': 17,
 'علم': 18,
 'يسي': 19,
 'ثلث': 20,
 'حدث': 21,
 'عكس': 22,
 '😂': 23,
 'خلو': 24,
 'ضغط': 25,
 'حيه': 26,
 'نعش': 27,
 'ارض': 28,
 'اعد': 29,
 'ولم': 30,
 'سلم': 31,
 'وكل': 32,
 'داي': 33,
 'وكن': 34,
 'وثق': 35,
 'بلل': 36,
 'وصل': 37,
 'بقل': 38,
 'جيت': 39,
 'قعد': 40,
 'معو': 41,
 'شو': 42,
 'تقم': 43,
 'نسي': 44,
 'شي': 45,
 '🤔': 46,
 'حيت': 47,
 'تعد': 48,
 'صحب': 49,
 'سمو': 50,
 'لكي': 51,
 'امر': 52,
 'دكتور': 53,
 'سعد': 54,
 'عبدالعزيز': 55,
 'طقه': 56,
 'جلس': 57,
 'دور': 58,
 'كثر': 59,
 'زعج': 60,
 'خرج': 61,
 'مثل': 62,
 'كون': 63,
 'فوز': 64,
 'علي': 65,
 'صعب': 66,
 'ازم': 67,
 'تحد': 68,
 'ايم': 69,
 'صبر': 70,
 'كفح': 71,
 'حمل': 72,
 'مجد': 73,
 'نجم': 74,
 'الف': 75,
 'برك': 76,
 'نخب': 77,
 'سور': 78,
 'كسب': 79,
 'حرم': 80,
 'جمع': 81,
 'برح': 82,
 'عزم': 83,
 'شكر': 84,
 'دار': 85

In [106]:
len(word_level_tokenizer.vocab)

1320

## Character Level Tokenization

In [107]:
sample_tokens = list(sample)
sample_tokens

['ا',
 'و',
 'م',
 ' ',
 'ب',
 'ا',
 'ن',
 ' ',
 'ا',
 'ن',
 'س',
 ' ',
 'ن',
 'ط',
 'ف',
 ' ',
 'ج',
 'م',
 'ل',
 ' ',
 'ب',
 'ع',
 'د',
 ' ',
 'ي',
 'ح',
 'ب',
 ' ',
 'ب',
 'ر',
 'ق',
 ' ',
 'ع',
 'ي',
 'ن',
 ' ',
 'خ',
 'ف',
 'ي',
 ' ',
 'ص',
 'ب',
 'ح',
 ' ',
 'ذ',
 'ب',
 'ل',
 ' ',
 'ن',
 'ط',
 'ف',
 ' ',
 'ت',
 'ح',
 'ل',
 ' ',
 'ر',
 'ب',
 'ع',
 ' ',
 'خ',
 'ر',
 'ف']

In [108]:
char_level_tokens = [token for tweet in tweets for token in tweet]

char_level_tokens = set(char_level_tokens)
print(len(char_level_tokens))

325


In [109]:
char2idx = {token: index for index, token in enumerate(char_level_tokens)}
char2idx

{'😂': 0,
 'ﻗ': 1,
 '🕖': 2,
 '💙': 3,
 '💔': 4,
 '🇮': 5,
 '🌸': 6,
 'ش': 7,
 '♀': 8,
 '🙏': 9,
 '🎂': 10,
 ' ': 11,
 'ت': 12,
 '😁': 13,
 'า': 14,
 '♓': 15,
 '😳': 16,
 'ظ': 17,
 '🎁': 18,
 '👍': 19,
 '🤥': 20,
 '💓': 21,
 '😑': 22,
 'ﺃ': 23,
 '🇸': 24,
 '😘': 25,
 '🦅': 26,
 'س': 27,
 '🤣': 28,
 'ﺎ': 29,
 '주': 30,
 '😇': 31,
 '🙊': 32,
 '니': 33,
 '🤗': 34,
 'د': 35,
 'ﺱ': 36,
 'ﻟ': 37,
 'ى': 38,
 '\u2069': 39,
 '🤯': 40,
 '✨': 41,
 '–': 42,
 '👮': 43,
 'ث': 44,
 'ط': 45,
 '😐': 46,
 '💤': 47,
 '💸': 48,
 '🤨': 49,
 'ۆ': 50,
 '⌛': 51,
 '🏿': 52,
 '💎': 53,
 '🌚': 54,
 '📍': 55,
 '😷': 56,
 'ﻓ': 57,
 'ﺍ': 58,
 '🚶': 59,
 'ﺮ': 60,
 '😸': 61,
 '⸀': 62,
 'ه': 63,
 'ﺖ': 64,
 '💡': 65,
 'ﻛ': 66,
 '٪': 67,
 '💭': 68,
 '😭': 69,
 '•': 70,
 '😊': 71,
 '⭕': 72,
 '어': 73,
 '🌥': 74,
 'แ': 75,
 'ﻌ': 76,
 '🏾': 77,
 'ف': 78,
 'چ': 79,
 '✅': 80,
 '😒': 81,
 '🇯': 82,
 '🏻': 83,
 'و': 84,
 'ء': 85,
 'ن': 86,
 'ﻱ': 87,
 '🏽': 88,
 '😠': 89,
 '😪': 90,
 '☺': 91,
 '😎': 92,
 '🇲': 93,
 '·': 94,
 '🇴': 95,
 '￼': 96,
 '🤧': 97,
 '😴': 98,
 'ﻳ': 99,
 '»':

In [ ]:
# TODO: Implement character level tokenizer
# 1. __init__()
# 2. _create_vocab
# 3. _tokenize_document
class CharacterLevelTokenizer(Tokenizer):
    def __init__(self, corpus: list[str], min_frequency: int = 2):
        super().__init__(corpus, min_frequency)

    def _create_vocab(self, corpus: list[str]) -> dict[str, int]:
        tokens_counter = Counter([token for sample in corpus for token in sample])
        tokens = [
            token
            for token, count in tokens_counter.items()
            if count >= self.min_frequency
        ]


        # Filter out characters below min_frequency
        return  {token: index for index, token in enumerate(tokens, start=self.min_frequency)}

    def _tokenize_document(self, doc: str) -> list[str]:
        return [self.vocab.get(token, -1) for token in doc]

In [ ]:
char_level_tokenizer = CharacterLevelTokenizer(corpus=tweets, min_frequency=0)


In [127]:
tweets[0],len(char_level_tokenizer._tokenize_document(tweets[0]))


('اوم بان انس نطف جمل بعد يحب برق عين خفي صبح ذبل نطف تحل ربع خرف', 63)

> Notice the difference in the vocabulary size, between word level and character level. Why would you choose one over the other? 

## Result of Tokenization

1. List of documents (corpus)
2. Each document is represented by a sequence of tokens

> Not all documents have the same length

In [119]:
tokenized_tweets = word_level_tokenizer.tokenize(tweets)
char_tokenized_tweets = char_level_tokenizer.tokenize(tweets)

In [129]:
max([len(t) for t in tokenized_tweets]), max([len(t) for t in char_tokenized_tweets])

(82, 163)

# n-grams

To be continued

# Save the Tokenizer

One of the most straight forward ways of saving a Python object is through binary `serialization`

serialization is a method of converting the `object` to `bytes`, these `bytes` can be read later to recreate the object

`pickle` package is the built-in package for object serialization

In [114]:
word_level_tokenizer_path = os.path.join(data_dir, "word-tokenizer.pkl")
word_level_vocab_path = os.path.join(data_dir, "word-level-vocab.json")

with open(word_level_tokenizer_path, "wb+") as f:
    pickle.dump(obj=word_level_tokenizer, file=f)

    
with open(word_level_vocab_path, "wt+") as f:
    json.dump(word_level_tokenizer.vocab, f)

In [115]:
# TODO: Pickle character level tokenizer

char_level_tokenizer_path = os.path.join(data_dir, "char-tokenizer.pkl")
char_level_vocab_path = os.path.join(data_dir, "char-level-vocab.json")

with open(char_level_tokenizer_path, "wb+") as f:
    pickle.dump(obj=char_level_tokenizer, file=f)

with open(char_level_vocab_path, "wt+") as f:
    json.dump(char_level_tokenizer.vocab, f)